In [1]:
import _section2_utils as section2_utils
import geopandas as gpd
import numpy as np
import pandas as pd
from IPython.display import HTML, Image, Markdown, display, display_html
from segment_speed_utils import gtfs_schedule_wrangling, helpers, time_series_utils
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)
from shared_utils import catalog_utils, rt_dates, rt_utils

In [2]:
import datetime

from dask import compute, delayed

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
sf = "City and County of San Francisco"

In [5]:
lawndale = "City of Lawndale"

In [6]:
la_metro = "Los Angeles County Metropolitan Transportation Authority"

In [7]:
start = datetime.datetime.now()
df = section2_utils.load_schedule_vp_metrics(la_metro)
end = datetime.datetime.now()

In [8]:
start = datetime.datetime.now()
df_lawndale = section2_utils.load_schedule_vp_metrics(lawndale)
end = datetime.datetime.now()

In [9]:
df.head(2)

,schedule_gtfs_dataset_key,dir_0_1,Period,Average Scheduled Service (trip minutes),Average Stop Distance (miles),# scheduled trips,Trips per Hour,is_express,is_rapid,is_rail,is_coverage,is_downtown_local,is_local,Date,Route typology,# Minutes with 1+ VP per Minute,# Minutes with 2+ VP per Minute,Aggregate Actual Service Minutes,Aggregate Scheduled Service Minutes (all trips),# VP,# VP within Scheduled Shape,# Early Arrival Trips,# On-Time Trips,# Late Trips,# Trips with VP,Average VP per Minute,% VP within Scheduled Shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,% Scheduled Trip w/ 1+ VP/Minute,% Scheduled Trip w/ 2+ VP/Minute,Realtime versus Scheduled Service Ratio,Average Actual Service (Trip Minutes),GTFS Availability,Speed (MPH),Transit Operator,route_long_name,route_short_name,Route,Route ID,schedule_source_record_id,Base64 Encoded Feed URL,Organization ID,Organization,District,ruler_100_pct,ruler_for_vp_per_min,frequency_in_minutes,direction_id,Direction,service_date
0,2a0571758141f412b6a546fd70a65bf3,0.00,all_day,53.20,0.98,97,4.04,0.00,0.00,1.00,0.00,1.00,0.00,2023-03-15,rail,5824,4265,8434.44,5160.00,12876,11940,0,4,93,97,1.53,93.00,69.00,51.00,100.00,83.00,1.63,86.95,schedule_and_vp,22.39,LA Metro Rail Schedule,Metro A Line (Blue),,Metro A Line,801,recofCmylEKq2zuPr,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX3JhaWwvcmF3L21hc3Rlci9ndGZzX3JhaWwuemlw,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation Authority,07 - Los Angeles,100,2,14.85,0.00,Northbound,2023-03-15
1,2a0571758141f412b6a546fd70a65bf3,0.00,all_day,115.71,0.98,94,3.92,0.00,0.00,1.00,0.00,1.00,0.00,2023-07-12,rail,4388,3123,11879.46,5149.00,9413,9370,11,1,33,45,0.79,100.00,37.00,26.00,85.00,61.00,2.31,263.99,schedule_and_vp,23.16,LA Metro Rail Schedule,Metro A-Line,,Metro A Line,801,recofCmylEKq2zuPr,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX3JhaWwvcmF3L21hc3Rlci9ndGZzX3JhaWwuemlw,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation Authority,07 - Los Angeles,100,2,15.31,0.00,Northbound,2023-07-12


In [10]:
df.dir_0_1.value_counts().sum()

9767

In [11]:
len(df)

9767

In [12]:
df.Direction.value_counts()

Northbound    2587
Southbound    2543
Westbound     2319
Eastbound     2318
Name: Direction, dtype: int64

In [13]:
df.Direction.value_counts().sum()

9767

In [14]:
df.head(1)

,schedule_gtfs_dataset_key,dir_0_1,Period,Average Scheduled Service (trip minutes),Average Stop Distance (miles),# scheduled trips,Trips per Hour,is_express,is_rapid,is_rail,is_coverage,is_downtown_local,is_local,Date,Route typology,# Minutes with 1+ VP per Minute,# Minutes with 2+ VP per Minute,Aggregate Actual Service Minutes,Aggregate Scheduled Service Minutes (all trips),# VP,# VP within Scheduled Shape,# Early Arrival Trips,# On-Time Trips,# Late Trips,# Trips with VP,Average VP per Minute,% VP within Scheduled Shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,% Scheduled Trip w/ 1+ VP/Minute,% Scheduled Trip w/ 2+ VP/Minute,Realtime versus Scheduled Service Ratio,Average Actual Service (Trip Minutes),GTFS Availability,Speed (MPH),Transit Operator,route_long_name,route_short_name,Route,Route ID,schedule_source_record_id,Base64 Encoded Feed URL,Organization ID,Organization,District,ruler_100_pct,ruler_for_vp_per_min,frequency_in_minutes,direction_id,Direction,service_date
0,2a0571758141f412b6a546fd70a65bf3,0.00,all_day,53.20,0.98,97,4.04,0.00,0.00,1.00,0.00,1.00,0.00,2023-03-15,rail,5824,4265,8434.44,5160.00,12876,11940,0,4,93,97,1.53,93.00,69.00,51.00,100.00,83.00,1.63,86.95,schedule_and_vp,22.39,LA Metro Rail Schedule,Metro A Line (Blue),,Metro A Line,801,recofCmylEKq2zuPr,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX3JhaWwvcmF3L21hc3Rlci9ndGZzX3JhaWwuemlw,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation Authority,07 - Los Angeles,100,2,14.85,0.00,Northbound,2023-03-15


In [15]:
# df["Route"].unique()

In [16]:
df.groupby(["Route", "dir_0_1"]).agg({"Direction": "nunique"}).sort_values(
    by=["Direction"], ascending=False
).head()

Direction
Route                  dir_0_1           
256 Metro Local Line   0.00             2
237 Metro Local Line   0.00             2
                       1.00             2
256 Metro Local Line   1.00             2
344 Metro Limited Line 0.00             1

## Fix Graphs

In [17]:
# Data Dictionary
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")
import yaml

with open("readable.yml") as f:
    readable_dict = yaml.safe_load(f)

# Color Palette
with open("color_palettes.yml") as f:
    color_dict = yaml.safe_load(f)

In [18]:
import altair as alt

alt.data_transformers.enable("default", max_rows=None)
import _report_utils

In [19]:
# Set drop down menu to be on the upper right
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

In [20]:
routes_list = df["Route"].unique().tolist()

route_dropdown = alt.binding_select(
    options=routes_list,
    name="Routes: ",
)
# Column that controls the bar charts
xcol_param = alt.selection_point(
    fields=["Route"], value=routes_list[0], bind=route_dropdown
)

### Function

In [21]:
import warnings

warnings.filterwarnings("ignore")

In [22]:
# section2_utils.filtered_route(df)

In [23]:
# section2_utils.filtered_route(df_lawndale)

### Timeliness Trips
* The title was displaying the wrong value. For example, the chart/tooltip showed the route going Northbound but the title showed "Eastbound." I think the issue arises from the fact that the Altair dropdown menu to control the graphs does not filter the actual dataframe. My original function grabbed the first available value for the cardinal direction, which does not match with the route selected in the route dropdown menu. 

In [24]:
timeliness_df = section2_utils.timeliness_trips(df)

In [25]:
timeliness_df.head(2)

,Date,Organization,Route,Period,Direction,dir_0_1,variable,value
0,2023-03-15,Los Angeles County Metropolitan Transportation Authority,Metro A Line,offpeak,Northbound,0.00,# Early Arrival Trips,0
1,2023-07-12,Los Angeles County Metropolitan Transportation Authority,Metro A Line,offpeak,Northbound,0.00,# Early Arrival Trips,3


In [26]:
def base_facet_chart(
    df: pd.DataFrame,
    direction_to_filter: int,
    y_col: str,
    color_col: str,
    facet_col: str,
    title: str,
    subtitle: str,
):
    tooltip_cols = [
        "Period",
        "Route",
        "Organization",
        "Date",
        "Direction",
        y_col,
        color_col,
    ]

    try:
        title = title + " for Direction " + str(direction_to_filter)
    except:
        pass

    max_y = section2_utils.set_y_axis(df, y_col)
    df = section2_utils.clean_data_charts(df, y_col)

    chart = (
        alt.Chart(df)
        .mark_bar(size=7, clip=True)
        .encode(
            x=alt.X(
                "yearmonthdate(Date):O",
                title=["Date"],
                axis=alt.Axis(labelAngle=-45, format="%b %Y"),
            ),
            y=alt.Y(
                f"{y_col}:Q",
                title=_report_utils.labeling(y_col),
                scale=alt.Scale(domain=[0, max_y]),
            ),
            color=alt.Color(
                f"{color_col}:N",
                title=_report_utils.labeling(color_col),
                scale=alt.Scale(range=color_dict["red_green_yellow"]),
            ),
            tooltip=tooltip_cols,
        )
    )
    chart = chart.properties(width=200, height=250)
    chart = chart.facet(column=alt.Column(f"{facet_col}:N",)).properties(
        title={
            "text": title,
            "subtitle": subtitle,
        }
    )

    return chart

In [27]:
(
    (
        base_facet_chart(
            timeliness_df.loc[timeliness_df["dir_0_1"] == 0],
            0,
            "value",
            "variable",
            "Period",
            readable_dict["timeliness_trips_graph"]["title"],
            readable_dict["timeliness_trips_graph"]["subtitle"],
        )
    )
    .add_params(xcol_param)
    .transform_filter(xcol_param)
)

alt.FacetChart(...)

In [ ]:
(
    (
        base_facet_chart(
            timeliness_df.loc[timeliness_df["dir_0_1"] == 1],
            1,
            "value",
            "variable",
            "Period",
            readable_dict["timeliness_trips_graph"]["title"],
            readable_dict["timeliness_trips_graph"]["subtitle"],
        )
    )
    .add_params(xcol_param)
    .transform_filter(xcol_param)
)

### Frequency Graphs
* Just gets completely messed up.

In [ ]:
def frequency_chart2(
    df: pd.DataFrame, direction_id: int, title: str, subtitle: str
):
    df["Frequency in Minutes"] = (
        "A trip going this direction comes every "
        + df.frequency_in_minutes.astype(int).astype(str)
        + " minutes"
    )
    df = section2_utils.clean_data_charts(df, "frequency_in_minutes")
    df = df.loc[df.dir_0_1 == direction_id].reset_index(drop=True)

    fixed_x_values = [0, 30, 60, 90, 120, 150, 180, 210, 240]

    color_scale = alt.Scale(
        domain=color_dict["freq_domain"], range=color_dict["freq_range"]
    )

    chart = (
        alt.Chart(df)
        .mark_bar(size=7, clip=True)
        .encode(
            y=alt.Y(
                "yearmonthdate(Date):O",
                title=["Date"],
                axis=alt.Axis(format="%b %Y"),
            ),
            x=alt.X(
                "frequency_in_minutes:Q",
                title=_report_utils.labeling("frequency_in_minutes"),
                scale=alt.Scale(domain=[0, 240]),
            ),
            color=alt.Color(
                "frequency_in_minutes:Q",
                scale=color_scale,
                title=_report_utils.labeling("frequency_in_minutes"),
            ),
            tooltip=["Date", "Route", "Frequency in Minutes", "Period", "Direction"],
        )
    )

    chart = chart.properties(width=150, height=150)

    title = title + " for Direction " + str(direction_id)
    chart = chart.facet(column=alt.Column("Period:N")).properties(
        title={
            "text": title,
            "subtitle": subtitle,
        }
    )
    return chart

In [ ]:
frequency_chart2(
    df,
    0,
    readable_dict["frequency_graph"]["title"],
    readable_dict["frequency_graph"]["subtitle"],
).add_params(xcol_param).transform_filter(xcol_param)

In [ ]:
frequency_chart2(
    df,
    1,
    readable_dict["frequency_graph"]["title"],
    "",
).add_params(xcol_param).transform_filter(xcol_param)

### Text

In [ ]:
route_stats_df = section2_utils.route_stats(df)

In [ ]:
def create_text_table(df: pd.DataFrame, direction: int):

    df = df.loc[df["Dir 0 1"] == direction].drop_duplicates().reset_index(drop=True)
    
    df2 = df.melt(
            id_vars=[
                "Route",
                "Direction",
            ],
            value_vars=[
                "Average Scheduled Service (Trip Minutes)",
                "Average Stop Distance (Miles)",
                "# Scheduled Trips",
                "Gtfs Availability",
                "Peak Avg Speed",
                "Peak Scheduled Trips",
                "Peak Hourly Freq",
                "Offpeak Avg Speed",
                "Offpeak Scheduled Trips",
                "Trips Per Hour",
            ],
        )
    # Create a decoy column to center all the text
    df2["Zero"] = 0

    df2["combo_col"] = df2.variable.astype(str) + ": " + df2.value.astype(str)
    df2.combo_col = df2.combo_col.str.replace(
            "schedule_and_vp", "Schedule and Realtime Data",
        ).str.replace("Gtfs", "GTFS")
    text_chart = (
            alt.Chart(df2)
            .mark_text()
            .encode(x=alt.X("Zero:Q", axis=None), y=alt.Y("combo_col", axis=None))
        )
    
    text_chart = text_chart.encode(text="combo_col:N").properties(
            title=f"Route Statistics for Direction {direction}",
            width=400,
            height=250,
        )
    return text_chart

In [ ]:
(create_text_table(route_stats_df,0).add_params(xcol_param)
    .transform_filter(xcol_param))